In [92]:
import lazypredict
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [113]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
df_X_richard = pd.read_csv("template_features.csv", index_col=0)
df_X_tim = pd.read_csv("full_waveform_features.csv", index_col=0)
df_X_hsv = pd.read_csv("hsv_features.csv", index_col=0)
df_X = pd.concat((df_X_tim,df_X_richard),axis=1)
df_X = pd.concat((df_X, df_X_hsv),axis=1)
# df_X = pd.concat((df_X, df_X_last),axis=1)
df_Y = pd.read_csv("data/Y_train.csv", index_col="id")

Remove highly correlated features

In [114]:
def rm_corr(X):
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    print("Removed columns: ", len(to_drop))
    # Drop features 
    X.drop(to_drop, axis=1, inplace=True)
    #X_test.drop(to_drop, axis=1, inplace=True)
    
    return X

In [115]:
df_X = rm_corr(df_X)

Removed columns:  23


Normalize

In [116]:
transformer = RobustScaler()
X = transformer.fit_transform(df_X)

In [117]:
X.shape

(5117, 76)

Impute missing values

In [118]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp_median.fit_transform(X)

Feature Selection

In [119]:
X_new = SelectKBest(f_classif, k=65).fit_transform(X, df_Y)
X_new.shape

(5117, 65)

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, df_Y,stratify=df_Y,test_size=.5,random_state=0)
clf = LazyClassifier(verbose=0, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:29<00:00,  1.01s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.82,0.68,None,0.81,2.21
XGBClassifier,0.82,0.68,None,0.81,4.21
LogisticRegression,0.79,0.68,None,0.78,0.18
LinearDiscriminantAnalysis,0.77,0.68,None,0.76,0.10
RandomForestClassifier,0.81,0.67,None,0.81,2.32
NearestCentroid,0.67,0.67,None,0.67,0.04
SGDClassifier,0.76,0.67,None,0.75,0.18
LinearSVC,0.78,0.67,None,0.77,2.11
GaussianNB,0.67,0.66,None,0.66,0.05


In [ ]:
# XGB, LGBM, RandomForestClassifier, ExtraTrees

In [121]:
df_X_hsv.shape

(5117, 22)